# Unit 5: Programming with the Structured API

## Contents
```
5.1 Structured data processing
5.2 SQL
5.3 DataFrames
5.4 Performance improvement
5.5 SparkSession
5.6 Creating DataFrames
5.7 Saving DataFrames
5.8 DataFrame operations
5.9 Columns, Expressions and DataFrame operations
5.10 Combining multiple conditions
5.11 Using the underlying RDD
5.12 Working with Pandas
5.13 SQL Queries
5.14 List of built-in functions
5.15 UDFs
```

## Structured data processing
Spark SQL is a Spark module for **structured data processing**.

Unlike the basic Spark RDD API, the interfaces provided by Spark SQL provide Spark with more information about the structure of both the data and the computation being performed. Internally, Spark SQL uses this extra information to **perform additional optimizations**.

There are several ways to interact with Spark SQL including **SQL, the DataFrames API and the Datasets API**. 

**When computing a result the same execution engine is used, independently of which API/language you are using to express the computation.** This unification means that developers can easily switch back and forth between the various APIs based on which provides the most natural way to express a given transformation.

## SQL
One use of Spark SQL is to **execute SQL queries written using either ANSI SQL syntax or HiveQL**. Spark SQL can also be used to read data from an existing Hive installation. For more on how to configure this feature, please refer to the Hive Tables section. When running SQL from within another programming language the results will be returned as a DataFrame. You can also interact with the SQL interface using the command-line or over JDBC/ODBC.

Reference: [Spark SQL, DataFrames and Datasets Guide](https://spark.apache.org/docs/2.4.0/sql-programming-guide.html)

## DataFrames
A DataFrame is a distributed collection of data **organized into named columns**. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs.

## Performance improvement
Spark SQL and DataFrames take advantadge of the fact that they are using structured data to optimize the performance using the [Catalyst query optimizer](https://databricks.com/blog/2015/04/13/deep-dive-into-spark-sqls-catalyst-optimizer.html).

![Performance comparison](https://databricks.com/wp-content/uploads/2015/02/Screen-Shot-2015-02-16-at-9.46.39-AM.png)
Reference: [Performance improvements in Spark](https://databricks.com/blog/2015/04/24/recent-performance-improvements-in-apache-spark-sql-python-dataframes-and-more.html)

![Catalyst Optimizer](https://bigdata.cesga.es/img/catalyst_optimizer.png)
Reference: [Spark: The Definitive Guide](https://github.com/databricks/Spark-The-Definitive-Guide)

## SparkSession
In Spark 2 to use Spark SQL or DataFrames we use a **SparkSession** object as the main entry point to the API, in a similar way as we use the SparkContext (sc) as the main entry point to the RDD API. In fact, inside the SparkSession object we have an attribute that points to the SparkContext.

Just be aware, that when using previous versions of Spark, to use Spark SQL or DataFrames instead of a **SparkSession** you have to use a **SQLContext** object, and that there are two implementations of the SQLContext object:
* SQLContext: basic
* HiveContext: more advanced
  * It is able to read Hive tables directly
  * Supports HiveQL language

Spark 2 unifies the SQLContext and HiveContext directly in the SparkSession object simplifying its usage.

In our case the Spark Shell provides us automatically with a SparkSession object called **spark**.

In [1]:
type(spark)

pyspark.sql.session.SparkSession

## Creating DataFrames

### Creating a DataFrame from an existing file in JSON format

Reading an existing file in **JSON** format:

In [2]:
processes = spark.read.json('datasets/pacct_20170701_c66.json')

In [3]:
processes.printSchema()

root
 |-- command: string (nullable = true)
 |-- etime: double (nullable = true)
 |-- exitcode: long (nullable = true)
 |-- flag: long (nullable = true)
 |-- gid: long (nullable = true)
 |-- host: string (nullable = true)
 |-- io: long (nullable = true)
 |-- majflt: long (nullable = true)
 |-- mem: long (nullable = true)
 |-- minflt: long (nullable = true)
 |-- pid: long (nullable = true)
 |-- ppid: long (nullable = true)
 |-- rw: long (nullable = true)
 |-- stime: double (nullable = true)
 |-- swaps: long (nullable = true)
 |-- tm_hour: long (nullable = true)
 |-- tm_isdst: long (nullable = true)
 |-- tm_mday: long (nullable = true)
 |-- tm_min: long (nullable = true)
 |-- tm_mon: long (nullable = true)
 |-- tm_sec: long (nullable = true)
 |-- tm_wday: long (nullable = true)
 |-- tm_yday: long (nullable = true)
 |-- tm_year: long (nullable = true)
 |-- tty: long (nullable = true)
 |-- uid: long (nullable = true)
 |-- utime: double (nullable = true)
 |-- version: long (nullable = true)

### Creating a DataFrame from an existing file in Parquet format

Reading an existing file in **parquet** format:

In [4]:
processes2 = spark.read.parquet('datasets/pacct_20170701.parquet')

In [5]:
processes2.printSchema()

root
 |-- host: string (nullable = true)
 |-- flag: integer (nullable = true)
 |-- version: integer (nullable = true)
 |-- tty: integer (nullable = true)
 |-- exitcode: integer (nullable = true)
 |-- uid: integer (nullable = true)
 |-- gid: integer (nullable = true)
 |-- pid: integer (nullable = true)
 |-- ppid: integer (nullable = true)
 |-- tm_year: integer (nullable = true)
 |-- tm_mon: integer (nullable = true)
 |-- tm_mday: integer (nullable = true)
 |-- tm_hour: integer (nullable = true)
 |-- tm_min: integer (nullable = true)
 |-- tm_sec: integer (nullable = true)
 |-- tm_wday: integer (nullable = true)
 |-- tm_yday: integer (nullable = true)
 |-- tm_isdst: integer (nullable = true)
 |-- etime: decimal(10,2) (nullable = true)
 |-- utime: decimal(10,2) (nullable = true)
 |-- stime: decimal(10,2) (nullable = true)
 |-- mem: integer (nullable = true)
 |-- io: integer (nullable = true)
 |-- rw: integer (nullable = true)
 |-- minflt: integer (nullable = true)
 |-- majflt: integer (nul

### Creating a DataFrame from an existing file in CSV format

Reading an existing file in **CSV** format:

```
[jlopez@cdh61-login8 NYC_taxi_trip_records]$ head yellow_tripdata_2018-12.csv 
VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount

1,2018-12-01 00:28:22,2018-12-01 00:44:07,2,2.50,1,N,148,234,1,12,0.5,0.5,3.95,0,0.3,17.25
1,2018-12-01 00:52:29,2018-12-01 01:11:37,3,2.30,1,N,170,144,1,13,0.5,0.5,2.85,0,0.3,17.15
2,2018-12-01 00:12:52,2018-12-01 00:36:23,1,.00,1,N,113,193,2,2.5,0.5,0.5,0,0,0.3,3.8
1,2018-12-01 00:35:08,2018-12-01 00:43:11,1,3.90,1,N,95,92,1,12.5,0.5,0.5,2.75,0,0.3,16.55
```

In [6]:
trips = spark.read.csv('datasets/NYC_taxi_trip_records')

In [7]:
trips.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)



In order to use the appropriate column names we must indicate that the first line of the file contains a header with column names:

In [8]:
trips = spark.read.csv('datasets/NYC_taxi_trip_records', header=True)

In [9]:
trips.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)



To infer automatically the column data types:

In [10]:
trips = spark.read.csv('datasets/NYC_taxi_trip_records', header=True,
                       inferSchema=True)

In [11]:
trips.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



And finally, in order to be sure that the date values are parsed correctly we must specify the datetime format used: 

In [12]:
trips = spark.read.csv('datasets/NYC_taxi_trip_records', header=True,
                       inferSchema=True, timestampFormat="yyyy-MM-dd HH:mm:ss")

In [13]:
trips.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



### Creating a DataFrame from an existing file in other formats

Other formats like Avro, HBase, etc. are also supported, in some cases using third party data sources.

### Creating a DataFrame from a Hive table

In [14]:
trips = spark.sql('select * from demos.trips limit 1000')

In [15]:
trips.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [16]:
trips.count()

1000

### Creating a DataFrame from an RDD

A DataFrame is built from **an RDD that has a collection of Row objects** using the toDF() function.

In [17]:
from pyspark.sql import Row

In [18]:
peopleRDD = sc.parallelize([('Aroa', 18, 'student'), ('Lara', 15, 'student'), ('Susana', 35, 'teacher')])

In [19]:
type(peopleRDD)

pyspark.rdd.RDD

In this case we have to convert the collection of tuples in a collection of Rows and then we can transform it in an DF.

In [20]:
peopleDF = peopleRDD.map(lambda p: Row(name=p[0], age=int(p[1]), profession=p[2])).toDF()

In [21]:
type(peopleDF)

pyspark.sql.dataframe.DataFrame

In [22]:
peopleDF.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)
 |-- profession: string (nullable = true)



## Saving a DataFrame

In [23]:
trips.write.parquet('trips_parquet')

It creates a directory in HDFS and stores there the data using one file per partition using parquet format:
```bash
[jlopez@cdh61-login5 ~]$ hdfs dfs -ls trips_parquet
Found 2 items
-rw-r--r--   3 jlopez cesga          0 2020-04-30 12:13 trips_parquet/_SUCCESS
-rw-r--r--   3 jlopez cesga      31475 2020-04-30 12:13 trips_parquet/part-00000-b5c8ea86-7247-4ac3-b8d3-fdf477f65274-c000.snappy.parquet
```

We can also use other output formats like JSON, CSV or Avro:

In [24]:
trips.write.json('trips_json')

```bash
[jlopez@cdh61-login5 ~]$ hdfs dfs -ls trips_json
Found 2 items
-rw-r--r--   3 jlopez cesga          0 2020-04-30 12:14 trips_json/_SUCCESS
-rw-r--r--   3 jlopez cesga     390517 2020-04-30 12:14 trips_json/part-00000-783c9be0-4925-42a8-b742-7ffa84d9053e-c000.json
```

In [25]:
trips.write.csv('trips_csv')

```bash
[jlopez@cdh61-login5 ~]$ hdfs dfs -ls trips_csv
Found 2 items
-rw-r--r--   3 jlopez cesga          0 2020-04-30 12:14 trips_csv/_SUCCESS
-rw-r--r--   3 jlopez cesga     112517 2020-04-30 12:14 trips_csv/part-00000-15fdc03a-67ea-4127-a427-a5e525367234-c000.csv
```

In [26]:
trips.write.format('avro').save('trips_avro')

```bash
[jlopez@cdh61-login5 ~]$ hdfs dfs -ls trips_avro
Found 2 items
-rw-r--r--   3 jlopez cesga          0 2020-04-30 12:14 trips_avro/_SUCCESS
-rw-r--r--   3 jlopez cesga      38912 2020-04-30 12:14 trips_avro/part-00000-506db0eb-8975-4463-a20b-5e214172df53-c000.avro
```

We can even export the dataframe back to Hive as a Hive table, for example to export it to a table in the `jlopez` database:

In [27]:
trips.write.saveAsTable('jlopez.trips')

Then we can query that table directly from Hive.

## DataFrame Operations

As in the case of RDDs where we had transformations and actions in this case we also have them:
* Transformations (aka Queries): **lazy** transformations that create a new DataFrame
* Actions: trigger the execution of queries and return the data to the driver

### Actions

#### show

Displays the first n rows

In [28]:
peopleDF.show(2)

+---+----+----------+
|age|name|profession|
+---+----+----------+
| 18|Aroa|   student|
| 15|Lara|   student|
+---+----+----------+
only showing top 2 rows



#### take

Returns the first n rows

In [29]:
peopleDF.take(2)

[Row(age=18, name=u'Aroa', profession=u'student'),
 Row(age=15, name=u'Lara', profession=u'student')]

#### collect

Returns all rows

In [30]:
peopleDF.collect()

[Row(age=18, name=u'Aroa', profession=u'student'),
 Row(age=15, name=u'Lara', profession=u'student'),
 Row(age=35, name=u'Susana', profession=u'teacher')]

#### count

In [31]:
peopleDF.count()

3

### Transformations (aka Queries)

#### distinct

In [32]:
df = spark.createDataFrame([Row(name='aroa', age=17), Row(name='aroa', age=17), Row(name='lara', age=14)])

In [33]:
df.show()

+---+----+
|age|name|
+---+----+
| 17|aroa|
| 17|aroa|
| 14|lara|
+---+----+



In [34]:
df.distinct().show()

+---+----+
|age|name|
+---+----+
| 14|lara|
| 17|aroa|
+---+----+



#### limit

In [35]:
processes.count()

832119

In [36]:
processes_chunk = processes.limit(100)

In [37]:
processes_chunk.count()

100

#### where/filter

The **where** and **filter** operations are equivalent: where() is an alias for filter().

In [38]:
peopleDF.where('name = "Aroa"').show()

+---+----+----------+
|age|name|profession|
+---+----+----------+
| 18|Aroa|   student|
+---+----+----------+



The query is expressed using a Query String (see Query Strings section below).

#### select

We can do projections, for example reducing the number of columns:

In [39]:
processes.select('pid', 'command', 'etime')

DataFrame[pid: bigint, command: string, etime: double]

We can also do transformations (see Column Expressions section below):

In [40]:
from pyspark.sql.functions import col, expr

In [41]:
peopleDF.select(col('name'), col('age') + 100).show()

+------+-----------+
|  name|(age + 100)|
+------+-----------+
|  Aroa|        118|
|  Lara|        115|
|Susana|        135|
+------+-----------+



and we can also write it using expressions:

In [42]:
peopleDF.select(expr('name'), expr('age + 100')).show()

+------+-----------+
|  name|(age + 100)|
+------+-----------+
|  Aroa|        118|
|  Lara|        115|
|Susana|        135|
+------+-----------+



#### selectExpr

It is so common to use expressions as arguments when calling select like in the previous example:

In [43]:
peopleDF.select(expr('name'), expr('age + 100')).show()

+------+-----------+
|  name|(age + 100)|
+------+-----------+
|  Aroa|        118|
|  Lara|        115|
|Susana|        135|
+------+-----------+



that there is a convenient method that allows to indicate just expressions as arguments to the select:

In [44]:
peopleDF.selectExpr('name', 'age + 100').show()

+------+-----------+
|  name|(age + 100)|
+------+-----------+
|  Aroa|        118|
|  Lara|        115|
|Susana|        135|
+------+-----------+



So we can easily write also aggregation queries (you can find more details on aggregations are given in the groupBy section):

In [45]:
peopleDF.selectExpr('max(age)').show()

+--------+
|max(age)|
+--------+
|      35|
+--------+



that would be equivalent to the following:

In [46]:
from pyspark.sql.functions import max
peopleDF.select(max(col('age'))).show()

+--------+
|max(age)|
+--------+
|      35|
+--------+



In [47]:
peopleDF.select('age').groupBy().max().show()

+--------+
|max(age)|
+--------+
|      35|
+--------+



#### orderBy/sort

The **orderBy** and **sort** operations are equivalent: orderBy() is an alias for sort().

In [48]:
peopleDF.orderBy(col('age').desc()).show()

+---+------+----------+
|age|  name|profession|
+---+------+----------+
| 35|Susana|   teacher|
| 18|  Aroa|   student|
| 15|  Lara|   student|
+---+------+----------+



The order is controlled by a Column Expression: .asc() and .desc() are column expressions (see the Column Expressions section below).

#### groupBy

We can group data (it returns a GroupedData object with additional operations):

In [49]:
peopleDF.groupBy('profession')

And then we can perform operations on grouped data:

* Calculate the maximum/minimum:

In [50]:
peopleDF.groupBy('profession').max('age').show()

+----------+--------+
|profession|max(age)|
+----------+--------+
|   teacher|      35|
|   student|      18|
+----------+--------+



* Calculate the mean:

In [51]:
peopleDF.groupBy('profession').mean('age').show()

+----------+--------+
|profession|avg(age)|
+----------+--------+
|   teacher|    35.0|
|   student|    16.5|
+----------+--------+



* Calculate the sum:

In [52]:
peopleDF.groupBy('profession').sum('age').show()

+----------+--------+
|profession|sum(age)|
+----------+--------+
|   teacher|      35|
|   student|      33|
+----------+--------+



Reference: [Available GroupedData operations](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html#pyspark.sql.GroupedData)

#### join

We can join DataFrames in a similar way as we did with PairRDDs:

In [53]:
professionsDF = spark.createDataFrame([Row(name='student', description='A person engaged in study'), Row(name='teacher', description='A person whose occupation is teaching')])

In [54]:
peopleDF.join(professionsDF, peopleDF.profession == professionsDF.name).show()

+---+------+----------+--------------------+-------+
|age|  name|profession|         description|   name|
+---+------+----------+--------------------+-------+
| 35|Susana|   teacher|A person whose oc...|teacher|
| 18|  Aroa|   student|A person engaged ...|student|
| 15|  Lara|   student|A person engaged ...|student|
+---+------+----------+--------------------+-------+



#### withColumn

Adding columns to an existing DataFrame:

In [55]:
from pyspark.sql.functions import lit
peopleDF.withColumn('country', lit('spain')).show()

+---+------+----------+-------+
|age|  name|profession|country|
+---+------+----------+-------+
| 18|  Aroa|   student|  spain|
| 15|  Lara|   student|  spain|
| 35|Susana|   teacher|  spain|
+---+------+----------+-------+



#### when ... otherwise

We can also express conditions using the `when ... otherwise` construction:

In [56]:
from pyspark.sql.functions import when

peopleDF.withColumn('adult', when(col('age')<18, 'underage').otherwise('adult')).show()

+---+------+----------+--------+
|age|  name|profession|   adult|
+---+------+----------+--------+
| 18|  Aroa|   student|   adult|
| 15|  Lara|   student|underage|
| 35|Susana|   teacher|   adult|
+---+------+----------+--------+



### Columns, Expressions and DataFrame Operations

#### Columns

Some queries like select, sort, join or where can take column expressions.

In [57]:
# A column
peopleDF.name

Column<name>

In [58]:
# Another way to refer to the same column
peopleDF['name']

Column<name>

In [59]:
# Just another way to refer to the same column
col('name')

Column<name>

NOTE: In other to use the `col` function you have to import it from the functions module:

    from pyspark.sql.functions import col

We can operate on columns:

In [60]:
peopleDF.select(col('name'), col('age') + 100).show()

+------+-----------+
|  name|(age + 100)|
+------+-----------+
|  Aroa|        118|
|  Lara|        115|
|Susana|        135|
+------+-----------+



And we can give then as arguments:

In [61]:
peopleDF.orderBy(col('age').desc()).show()

+---+------+----------+
|age|  name|profession|
+---+------+----------+
| 35|Susana|   teacher|
| 18|  Aroa|   student|
| 15|  Lara|   student|
+---+------+----------+



In [62]:
peopleDF.orderBy(col('age').asc()).show()

+---+------+----------+
|age|  name|profession|
+---+------+----------+
| 15|  Lara|   student|
| 18|  Aroa|   student|
| 35|Susana|   teacher|
+---+------+----------+



In [63]:
peopleDF.where(col('profession').like('stu%')).show()

+---+----+----------+
|age|name|profession|
+---+----+----------+
| 18|Aroa|   student|
| 15|Lara|   student|
+---+----+----------+



#### Expressions

In fact, Columns are just expressions in its simplest form: `col('age')` is equivalent to `expr('age')`. But with expressions we can express more complex conditions using SQL:

In [64]:
from pyspark.sql.functions import expr

expr('age < 18')

Column<(age < 18)>

In general, **you can write your operations as DataFrame code or as SQL expressions (both are equivalent)**.

In cases like `select` we have seen that we have a special method called `selectExpr` that receives text as expressions. In other methods like `where` we can directly give as input expressions, dataframe column operations or text:

In [65]:
peopleDF.where('age > 10').show()

+---+------+----------+
|age|  name|profession|
+---+------+----------+
| 18|  Aroa|   student|
| 15|  Lara|   student|
| 35|Susana|   teacher|
+---+------+----------+



In [66]:
peopleDF.where(expr('age > 10')).show()

+---+------+----------+
|age|  name|profession|
+---+------+----------+
| 18|  Aroa|   student|
| 15|  Lara|   student|
| 35|Susana|   teacher|
+---+------+----------+



In [67]:
peopleDF.where(col('age') > 10).show()

+---+------+----------+
|age|  name|profession|
+---+------+----------+
| 18|  Aroa|   student|
| 15|  Lara|   student|
| 35|Susana|   teacher|
+---+------+----------+



In [68]:
peopleDF.where(peopleDF.age > 10).show()

+---+------+----------+
|age|  name|profession|
+---+------+----------+
| 18|  Aroa|   student|
| 15|  Lara|   student|
| 35|Susana|   teacher|
+---+------+----------+



In [69]:
peopleDF.where(peopleDF['age'] > 10).show()

+---+------+----------+
|age|  name|profession|
+---+------+----------+
| 18|  Aroa|   student|
| 15|  Lara|   student|
| 35|Susana|   teacher|
+---+------+----------+



### Combining multiple conditions

To combine conditions, instead of the standard Python keywords `and`, `or` or `not` you have to use their Spark equivalents **& | ~**:
* and -> **&**
* or -> **|**
* not -> **~**

This way you can combine conditions:

In [70]:
peopleDF.where((col('age') > 15) & col('profession').like('stud%')).show()

+---+----+----------+
|age|name|profession|
+---+----+----------+
| 18|Aroa|   student|
+---+----+----------+



In [71]:
peopleDF.where(col('name').like('Aroa') | col('name').like('Lara')).show()

+---+----+----------+
|age|name|profession|
+---+----+----------+
| 18|Aroa|   student|
| 15|Lara|   student|
+---+----+----------+



In [72]:
peopleDF.where(~col('profession').like('teacher')).show()

+---+----+----------+
|age|name|profession|
+---+----+----------+
| 18|Aroa|   student|
| 15|Lara|   student|
+---+----+----------+



## Using the underlying RDD

Sometimes it is useful to use a DataFrame as an RDD so you all the flexibility of the RDD API.

It is very easy to access the underlying RDD of Rows, it is exposed under the **.rdd** property:

In [73]:
import math
peopleDF.rdd.map(lambda row: (row.name, math.sqrt(row.age))).collect()

[(u'Aroa', 4.242640687119285),
 (u'Lara', 3.872983346207417),
 (u'Susana', 5.916079783099616)]

## Working with Pandas

To simplify our work, we can easily change from Spark DataFrames to Pandas DataFrames and do the operations that we need with the most appropriate technology.

It is very easy to convert a Spark DataFrame to a Pandas DataFrame, but be careful because while the Spark DataFrame is distributed across the nodes of the Hadoop cluster, the Pandas DataFrame will be just in the node that is running the driver:

In [74]:
import pandas as pd

pandas_df = peopleDF.toPandas()

type(pandas_df)

pandas.core.frame.DataFrame

In [75]:
pandas_df

,age,name,profession
0,18,Aroa,student
1,15,Lara,student
2,35,Susana,teacher


We can also create a Spark DataFrame directly from a Pandas DataFrame:

In [76]:
pandas_df = pd.DataFrame({'name': ['Aroa', 'Lara', 'Susana'], 'age': [9, 6, 20]})

type(pandas_df)

pandas.core.frame.DataFrame

In [77]:
spark_df = spark.createDataFrame(pandas_df)

type(spark_df)

pyspark.sql.dataframe.DataFrame

In [78]:
spark_df.show()

+---+------+
|age|  name|
+---+------+
|  9|  Aroa|
|  6|  Lara|
| 20|Susana|
+---+------+



## SQL Queries

Spark 2.4 supports both ANSI-SQL as well as Hive QL queries.

For example the SELECT statement supports the following semantics:

```
SELECT [ALL|DISTINCT] named_expression[, named_expression, ...]
    FROM relation[, relation, ...]
    [lateral_view[, lateral_view, ...]]
    [WHERE boolean_expression]
    [aggregation [HAVING boolean_expression]]
    [ORDER BY sort_expressions]
    [CLUSTER BY expressions]
    [DISTRIBUTE BY expressions]
    [SORT BY sort_expressions]
    [WINDOW named_window[, WINDOW named_window, ...]]
    [LIMIT num_rows]

named_expression:
    : expression [AS alias]

relation:
    | join_relation
    | (table_name|query|relation) [sample] [AS alias]
    : VALUES (expressions)[, (expressions), ...]
          [AS (column_name[, column_name, ...])]

expressions:
    : expression[, expression, ...]

sort_expressions:
    : expression [ASC|DESC][, expression [ASC|DESC], ...]
```

To express conditions `case...when...then...end` staments are also supported.

The first thing we have to do before launching SQL queries is to **create a local temporary view of the DataFrame**:

In [79]:
peopleDF.createOrReplaceTempView('people')

After doing this we can start launching SQL queries against the DataFrame data just using the view name we have created:

In [80]:
spark.sql('SELECT * FROM people WHERE age > 20').show()

+---+------+----------+
|age|  name|profession|
+---+------+----------+
| 35|Susana|   teacher|
+---+------+----------+



In [81]:
spark.sql('''SELECT * FROM people WHERE name LIKE "Ar%"''').show()

+---+----+----------+
|age|name|profession|
+---+----+----------+
| 18|Aroa|   student|
+---+----+----------+



And this would be a more complex example where we use a `case...when...then...end` statement to create a new column:

In [82]:
spark.sql('''SELECT name, age,
  CASE WHEN age < 18 THEN 'No'
       WHEN age >= 18 THEN 'Yes'
  END AS adult
FROM people''').show()

+------+---+-----+
|  name|age|adult|
+------+---+-----+
|  Aroa| 18|  Yes|
|  Lara| 15|   No|
|Susana| 35|  Yes|
+------+---+-----+



Spark SQL also provides a bunch of functions that you can use:

```
SHOW FUNCTIONS
```
These functions can be used inside SQL statements or using the DataFrames API and you can even create your own functions. More details in the next two sections.

## List of built-in functions

There are several builtin functions that can be useful to operate on Columns:

* [List of built-in functions](http://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html#module-pyspark.sql.functions)

When we are in doubt about how to do some transformation it is useful to check this list before proceeding to use the underlying RDD directly. For example we have methods for:
* abs
* avg
* cos
* concat
* regexp_extract
* regexp_replace
* sum
* when
* otherwise
* lit

NOTE: These functions are inside the `pyspark.sql.functions` module, so to use them you have to import them first.

## UDFs

When there is no builtin function available, we can create our own user-defined functions (UDF) that will allow us to use custom python code to operate on Columns.

### Using UDFs with DataFrames

You can define your own UDF functions just decorating them with the `udf` decorator:

In [83]:
from pyspark.sql.functions import udf

@udf('int')
def is_adult(age):
    return 1 if age >= 18 else 0

Or, if you prefer, you can also define them explicitely:

In [84]:
from pyspark.sql.types import IntegerType
is_underage = udf(lambda n: 1 if n < 18 else 0, IntegerType())

After that you can use them to operate on columns:

In [85]:
peopleDF.select(col('name'), col('age'), is_adult(col('age')).alias('adult'), is_underage(col('age')).alias('underage')).show()

+------+---+-----+--------+
|  name|age|adult|underage|
+------+---+-----+--------+
|  Aroa| 18|    1|       0|
|  Lara| 15|    0|       1|
|Susana| 35|    1|       0|
+------+---+-----+--------+



### Using UDFs in SQL

To use the functions in SQL you have to register them first:

In [86]:
spark.udf.register('is_adult', is_adult)
spark.udf.register('is_underage', is_underage)

<function __main__.<lambda>>

Then you can just use them as in your SQL statements as normal functions:

In [87]:
spark.sql('''select name, age, is_adult(age) as adult, is_underage(age) as underage from people''').show()

+------+---+-----+--------+
|  name|age|adult|underage|
+------+---+-----+--------+
|  Aroa| 18|    1|       0|
|  Lara| 15|    0|       1|
|Susana| 35|    1|       0|
+------+---+-----+--------+



Reference: [UDFs in Python](https://docs.databricks.com/spark/latest/spark-sql/udf-python.html)


## Exercises
Review the documentation:
* [pyspark.sql documentation](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html)

Exercises:
* Unit 5 Working with meteorological data, now using DataFrames. You can also try to solve it using SQL.
* Unit 5 Sentiment Analysis: Review the Sentiment Analysis notebook that makes use of DataFrames and Spark ML